In [2]:
import os
import glob
import json
import random
import numpy as np
import pandas as pd

In [3]:
info = json.load(open('../meta_data/splitted.json'))

In [4]:
info

{'development': {'male': ['54',
   '51',
   '14',
   '03',
   '11',
   '22',
   '10',
   '40',
   '34',
   '45',
   '02',
   '17',
   '41',
   '55',
   '23',
   '05',
   '53',
   '21',
   '30',
   '44',
   '13',
   '20',
   '39',
   '09'],
  'female': ['26', '56', '59', '60', '58', '28']},
 'evaluation': {'male': ['31',
   '01',
   '33',
   '35',
   '08',
   '18',
   '16',
   '46',
   '50',
   '49',
   '27',
   '06',
   '15',
   '19',
   '48',
   '37',
   '42',
   '24',
   '04',
   '32',
   '38',
   '07',
   '29',
   '25'],
  'female': ['47', '57', '36', '43', '52', '12']}}

In [5]:
np.random.seed(2023)
random.seed(2023)

In [5]:
# WORD = os.listdir('./data/experiments')

In [17]:
LOOP = 1
WORD = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
DIR = '../new_data'
# WORD = os.listdir('./data/experiments/')
# WORD = ['0_1', '9_0', '0_3', '0_9', '3_0', '0_6', '6_0', '1_0']
WORD_EXP = [(3, 1), (1, 0), (1, 6), (1, 3), (1, 9), (3, 0), (3, 9), (3, 6), (9, 1), (0, 1), (6, 1), (9, 0), (9, 3), (9, 6), (0, 3), (0, 9), (0, 6), (6, 0), (6, 3), (6, 9)]

In [18]:
def prepare_data_experiment(root_dir,info):
    pattern = '%s/%s/%s/%s/%s'
    for word in WORD_EXP:
        background = {
        'file': [],
        'speaker': [],
        'gender': [],
        'word': [], 
        }
        pairs = {
            'eval': {
                'female':[],
                'male':[],
                'all':[],
            }
        }
        all_speakers = set(info['evaluation']['female'] + info['evaluation']['male'] + info['development']['female'] + info['development']['male'])
        folder_word = word
        for phase in info:
            speakers = info[phase]
            for gender in speakers:
                if phase == 'development':
                    for spkr in speakers[gender]:
                        path = os.path.join(root_dir,'experiments', folder_word , spkr)
                        for file in os.listdir(path):
                            background['file'].append(pattern %(root_dir,'experiments',folder_word, spkr, file))
                            background['speaker'].append(spkr)
                            background['gender'].append(gender)
                            background['word'].append(folder_word)
                else:
                    for spkr in speakers[gender]:
                        imposters = set(info['evaluation'][gender] + info['development'][gender]) - set([spkr])
                        imposter_files = []
                        for imp in imposters:
                            imp_file = glob.glob('new_data'+('/loop%s/*/%s/*.wav')%(2, imp)) + glob.glob(root_dir+('/experiments/*/%s/*.wav')%(imp))
                            imposter_files.extend(imp_file)

                        # Get all 
                        all_imposters = all_speakers - set([spkr])
                        all_imposter_files = []
                        for imp in all_imposters:
                            imp_file = glob.glob('new_data'+('/loop%s/*/%s/*.wav')%(2, imp)) + glob.glob(root_dir+('/experiments/*/%s/*.wav')%(imp))
                            all_imposter_files.extend(imp_file)

                        path = os.path.join(root_dir, 'experiments',folder_word , spkr)
                        files = os.listdir(path)
                        for file in files:
                            current_file = pattern % (root_dir,'experiments', folder_word, spkr, file)
                            # Get target correct
                            # Accept pairs
                            other_files = [f for f in files if f != file]
                            accept_pairs = [['1', current_file, pattern % (root_dir,'experiments', folder_word, spkr, o_file)] for o_file in other_files]

                            # Reject pairs
                            reject_pairs = []
                            all_reject_pairs = []
                            n_pairs = len(files)
                            n_target_wrong = n_pairs // 2
                            n_imposters = n_pairs - n_target_wrong
                            
                            tw_all =list(set(glob.glob('new_data'+'/loop%s/*/%s/*.wav' % (2, spkr)) +glob.glob(root_dir+'/%s/*/%s/*.wav' % ('experiments', spkr))) -
                                set(glob.glob(root_dir+'/%s/%s/%s/*.wav' % ('experiments', folder_word, spkr))))
                            tw_set = random.choices(tw_all, k=n_target_wrong)
                            for tw_sample in tw_set:
                                reject_pairs.append(['0', current_file, tw_sample])
                                all_reject_pairs.append(['0', current_file, tw_sample])

                            imposter_set = random.choices(imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample])
                            pairs['eval'][gender].extend([accept_pairs] + [reject_pairs])


                            
                            imposter_set = random.choices(all_imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                all_reject_pairs.append(['0', current_file, imposter_sample])
                            pairs['eval']['all'].extend([accept_pairs] + [all_reject_pairs])

                            
        df_bg = pd.DataFrame(background)
        df_bg.to_csv(os.path.join(root_dir,'%s/%s/development.csv'%('experiments',folder_word)), index=False)
        print(word)
        for phase in pairs:
            files = pairs[phase]
            for gender in files:
                fout = open(os.path.join(root_dir,'%s/%s/%s_%s_version_2.txt'%('experiments',folder_word,gender,phase)), 'w')
                for list_file in files[gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')
  


In [19]:
prepare_data_experiment(DIR, info)

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'tuple'

In [24]:
def prepare_data_stage2_2(root_dir,info,loop):
    pattern = 'loop%s/%s/%s/%s'
    all_speakers = set(info['evaluation']['female'] + info['evaluation']['male'] + info['development']['female'] + info['development']['male'])
    for word in WORD:
        background = {
        'file': [],
        'speaker': [],
        'gender': [],
        'word': [], 
        }
        pairs = {
            'eval': {
                'female':[],
                'male':[],
                'all':[],
            }
        }
        folder_word = word + ('_'+word)*(loop-1)
        for phase in info:
            speakers = info[phase]
            for gender in speakers:
                if phase == 'development':
                    for spkr in speakers[gender]:
                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        for file in os.listdir(path):
                            background['file'].append(pattern %(loop, folder_word, spkr, file))
                            background['speaker'].append(spkr)
                            background['gender'].append(gender)
                            background['word'].append(folder_word)
                else:
                    for spkr in speakers[gender]:
                        imposters = set(info['evaluation'][gender] + info['development'][gender]) - set([spkr])
                        imposter_files = []
                        for imp in imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/*/%s/*.wav')%(loop, imp))
                            imposter_files.extend(imp_file)

                        # Get all 
                        all_imposters = all_speakers - set([spkr])
                        all_imposter_files = []
                        for imp in all_imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/*/%s/*.wav')%(loop, imp))
                            all_imposter_files.extend(imp_file)

                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        files = os.listdir(path)
                        for file in files:
                            current_file = pattern % (loop, folder_word, spkr, file)
                            # Get target correct
                            # Accept pairs
                            other_files = [f for f in files if f != file]
                            accept_pairs = [['1', current_file, pattern % (loop, folder_word, spkr, o_file)] for o_file in other_files]

                            # Reject pairs
                            reject_pairs = []
                            all_reject_pairs = []
                            n_pairs = len(files)
                            n_target_wrong = n_pairs // 2
                            n_imposters = n_pairs - n_target_wrong
                            
                            tw_all =list(set(glob.glob(root_dir+'/loop%s/*/%s/*.wav' % (loop, spkr))) -
                                set(glob.glob(root_dir+'/loop%s/%s/%s/*.wav' % (loop, folder_word, spkr))))
                            tw_set = random.choices(tw_all, k=n_target_wrong)
                            for tw_sample in tw_set:
                                reject_pairs.append(['0', current_file, tw_sample[len(root_dir)+1:]])
                                all_reject_pairs.append(['0', current_file, tw_sample[len(root_dir)+1:]])
                            imposter_set = random.choices(imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])
                            pairs['eval'][gender].extend([accept_pairs] + [reject_pairs])

                            
                            imposter_set = random.choices(all_imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                all_reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])
                            pairs['eval']['all'].extend([accept_pairs] + [all_reject_pairs])


        df_bg = pd.DataFrame(background)
        df_bg.to_csv(os.path.join(root_dir,'loop%s/%s/development.csv'%(loop,folder_word)), index=False)
        print(folder_word)
        for phase in pairs:
            files = pairs[phase]
            for gender in files:
                fout = open(os.path.join(root_dir,'loop%s/%s/%s_%s_tw.txt'%(loop,folder_word,gender,phase)), 'w')
                for list_file in files[gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')
            
            # files = pairs[phase]['male'] + pairs[phase]['female']
            # fout = open(os.path.join(root_dir,'loop%s/%s/all_%s.txt'%(loop,folder_word,phase)), 'w')
            # for list_file in files:
            #     for file in list_file:
            #         fout.write(" ".join(file) + '\n')
        

        


In [25]:
prepare_data_stage2_2('../new_data', info, loop=1)

0
1
2
3
4
5
6
7
8
9


# Independent generation

In [9]:
def prepare_data_stage2_no_tw(root_dir,info,loop):
    pattern = 'loop%s/%s/%s/%s'
    all_speakers = set(info['evaluation']['female'] + info['evaluation']['male'] + info['development']['female'] + info['development']['male'])
    for word in WORD:
        background = {
        'file': [],
        'speaker': [],
        'gender': [],
        'word': [], 
        }
        pairs = {
            'eval': {
                'female':[],
                'male':[],
                'all':[],
            }
        }
        folder_word = word + ('_'+word)*(loop-1)
        for phase in info:
            speakers = info[phase]
            for gender in speakers:
                if phase == 'development':
                    for spkr in speakers[gender]:
                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        for file in os.listdir(path):
                            background['file'].append(pattern %(loop, folder_word, spkr, file))
                            background['speaker'].append(spkr)
                            background['gender'].append(gender)
                            background['word'].append(folder_word)
                else:
                    for spkr in speakers[gender]:
                        imposters = set(info['evaluation'][gender] + info['development'][gender]) - set([spkr])
                        imposter_correct_files = []
                        for imp in imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/%s/%s/*.wav')%(loop, folder_word,imp))
                            imposter_correct_files.extend(imp_file)

                        imposter_wrong_files = []
                        for imp in imposters:
                            imp_file = list(set(glob.glob(root_dir+'/loop%s/*/%s/*.wav' % (loop, imp))) -
                                set(glob.glob(root_dir+'/loop%s/%s/%s/*.wav' % (loop, folder_word, imp))))
                            imposter_wrong_files.extend(imp_file)


                        # Get all 
                        all_imposters = all_speakers - set([spkr])
                        all_imposter_correct_files = []
                        for imp in all_imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/%s/%s/*.wav')%(loop,folder_word, imp))
                            all_imposter_correct_files.extend(imp_file)
                        
                        all_imposter_wrong_files = []
                        for imp in all_imposters:
                            imp_file = list(set(glob.glob(root_dir+'/loop%s/*/%s/*.wav' % (loop, imp))) -
                                set(glob.glob(root_dir+'/loop%s/%s/%s/*.wav' % (loop, folder_word, imp))))
                            all_imposter_wrong_files.extend(imp_file)

                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        files = os.listdir(path)
                        for file in files:
                            current_file = pattern % (loop, folder_word, spkr, file)
                            # Get target correct
                            # Accept pairs
                            other_files = [f for f in files if f != file]
                            accept_pairs = [['1', current_file, pattern % (loop, folder_word, spkr, o_file)] for o_file in other_files]

                            # Reject pairs
                            reject_pairs = []
                            all_reject_pairs = []
                            n_pairs = len(files)
                            n_imposter_correct = n_pairs // 2
                            n_imposters_wrong = n_pairs - n_imposter_correct
                            
                            #  Imposter correct
                            imposter_set = random.choices(imposter_correct_files, k=n_imposter_correct)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])

                            # imposter wrong
                            imposter_set = random.choices(imposter_wrong_files, k=n_imposters_wrong)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])
                            pairs['eval'][gender].extend([accept_pairs] + [reject_pairs])

                            # All imp correct 
                            imposter_set = random.choices(all_imposter_correct_files, k=n_imposter_correct)
                            for imposter_sample in imposter_set:
                                all_reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])
                             # All imp wrong 
                            imposter_set = random.choices(all_imposter_wrong_files, k=n_imposters_wrong)
                            for imposter_sample in imposter_set:
                                all_reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])
                            pairs['eval']['all'].extend([accept_pairs] + [all_reject_pairs])


        df_bg = pd.DataFrame(background)
        # df_bg.to_csv(os.path.join(root_dir,'loop%s/%s/development.csv'%(loop,folder_word)), index=False)
        print(folder_word)
        for phase in pairs:
            files = pairs[phase]
            for gender in files:
                fout = open(os.path.join(root_dir,'loop%s/%s/%s_%s_no_tw.txt'%(loop,folder_word,gender,phase)), 'w')
                for list_file in files[gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')
            
            # files = pairs[phase]['male'] + pairs[phase]['female']
            # fout = open(os.path.join(root_dir,'loop%s/%s/all_%s.txt'%(loop,folder_word,phase)), 'w')
            # for list_file in files:
            #     for file in list_file:
            #         fout.write(" ".join(file) + '\n')
        

        


In [14]:
prepare_data_stage2_no_tw('new_data', info, loop=5)

0_0_0_0_0
1_1_1_1_1
2_2_2_2_2
3_3_3_3_3
4_4_4_4_4
5_5_5_5_5
6_6_6_6_6
7_7_7_7_7
8_8_8_8_8
9_9_9_9_9


In [9]:
def prepare_data_experiment_no_tw(root_dir,info):
    pattern = '%s/%s/%s/%s/%s'
    for word in WORD:
        background = {
        'file': [],
        'speaker': [],
        'gender': [],
        'word': [], 
        }
        pairs = {
            'eval': {
                'female':[],
                'male':[],
                'all':[],
            }
        }
        all_speakers = set(info['evaluation']['female'] + info['evaluation']['male'] + info['development']['female'] + info['development']['male'])
        folder_word = word
        for phase in info:
            speakers = info[phase]
            for gender in speakers:
                if phase == 'development':
                    for spkr in speakers[gender]:
                        path = os.path.join(root_dir,'experiments', folder_word , spkr)
                        for file in os.listdir(path):
                            background['file'].append(pattern %(root_dir,'experiments',folder_word, spkr, file))
                            background['speaker'].append(spkr)
                            background['gender'].append(gender)
                            background['word'].append(folder_word)
                else:
                    for spkr in speakers[gender]:
                        imposters = set(info['evaluation'][gender] + info['development'][gender]) - set([spkr])
                        imposter_correct_files = []
                        for imp in imposters:
                            imp_file = glob.glob(root_dir+('/experiments/%s/%s/*.wav')%(folder_word,imp))
                            imposter_correct_files.extend(imp_file)

                        imposter_wrong_files = []
                        for imp in imposters:
                            imp_file = glob.glob('new_data'+('/loop%s/*/%s/*.wav')%(2,imp)) + list(set(glob.glob(root_dir+'/%s/*/%s/*.wav' % ('experiments', spkr))) -
                                set(glob.glob(root_dir+'/%s/%s/%s/*.wav' % ('experiments', folder_word, spkr))))
                            imposter_wrong_files.extend(imp_file)

                        # Get all 
                        all_imposters = all_speakers - set([spkr])
                        all_imposter_correct_files = []
                        for imp in all_imposters:
                            imp_file = glob.glob(root_dir+('/experiments/%s/%s/*.wav')%(folder_word,imp))
                            all_imposter_correct_files.extend(imp_file)
                        
                        all_imposters = all_speakers - set([spkr])
                        all_imposter_wrong_files = []
                        for imp in all_imposters:
                            imp_file =  glob.glob('new_data'+('/loop%s/*/%s/*.wav')%(2,imp)) + list(set(glob.glob(root_dir+'/%s/*/%s/*.wav' % ('experiments', spkr))) -
                                set(glob.glob(root_dir+'/%s/%s/%s/*.wav' % ('experiments', folder_word, spkr))))
                            all_imposter_wrong_files.extend(imp_file)

                        path = os.path.join(root_dir, 'experiments',folder_word , spkr)
                        files = os.listdir(path)
                        for file in files:
                            current_file = pattern % (root_dir,'experiments', folder_word, spkr, file)
                            # Get target correct
                            # Accept pairs
                            other_files = [f for f in files if f != file]
                            accept_pairs = [['1', current_file, pattern % (root_dir,'experiments', folder_word, spkr, o_file)] for o_file in other_files]

                            # Reject pairs
                            reject_pairs = []
                            all_reject_pairs = []
                            n_pairs = len(files)
                            n_imposter_correct = n_pairs // 2
                            n_imposters_wrong = n_pairs - n_imposter_correct
                            
                            #  Imposter correct
                            imposter_set = random.choices(imposter_correct_files, k=n_imposter_correct)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample])

                            # imposter wrong
                            imposter_set = random.choices(imposter_wrong_files, k=n_imposters_wrong)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample])
                            pairs['eval'][gender].extend([accept_pairs] + [reject_pairs])

                            # All imp correct 
                            imposter_set = random.choices(all_imposter_correct_files, k=n_imposter_correct)
                            for imposter_sample in imposter_set:
                                all_reject_pairs.append(['0', current_file, imposter_sample])
                             # All imp wrong 
                            imposter_set = random.choices(all_imposter_wrong_files, k=n_imposters_wrong)
                            for imposter_sample in imposter_set:
                                all_reject_pairs.append(['0', current_file, imposter_sample])
                            pairs['eval']['all'].extend([accept_pairs] + [all_reject_pairs])

                            
        df_bg = pd.DataFrame(background)
        # df_bg.to_csv(os.path.join(root_dir,'%s/%s/development.csv'%('experiments',folder_word)), index=False)
        print(word)
        for phase in pairs:
            files = pairs[phase]
            for gender in files:
                fout = open(os.path.join(root_dir,'%s/%s/%s_%s_no_tw.txt'%('experiments',folder_word,gender,phase)), 'w')
                for list_file in files[gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')
  


In [10]:
prepare_data_experiment_no_tw(DIR, info)

9_1
9_6
1_6
1_9
9_0
6_3
0_9
6_1
3_9
9_3
1_0
3_1
1_3
0_1
0_3
6_9
6_0
0_6
3_0
3_6


In [ ]:
def prepare_data_stage2(root_dir,info,loop):
    pattern = 'loop%s/%s/%s/%s'
    for word in WORD:
        background = {
        'file': [],
        'speaker': [],
        'gender': [],
        'word': [], 
        }
        pairs = {
            'development': {
                'female':[],
                'male': [],
            },
            'evaluation': {
                'female':[],
                'male':[],
            }
        }
        folder_word = word + ('_'+word)*(loop-1)
        for phase in info:
            speakers = info[phase]
            for gender in speakers:
                if phase == 'development':
                    for spkr in speakers[gender]:
                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        for file in os.listdir(path):
                            background['file'].append(pattern %(loop, folder_word, spkr, file))
                            background['speaker'].append(spkr)
                            background['gender'].append(gender)
                            background['word'].append(folder_word)
                else:
                    for spkr in speakers[gender]:
                        imposters = set(info['development'][gender] )+ set(info['evaluation'][gender]) - set([spkr])
                        imposter_files = []
                        for imp in imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/*/%s/*.wav')%(loop, imp))
                            imposter_files.extend(imp_file)

                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        files = os.listdir(path)
                        for file in files:
                            current_file = pattern % (loop, folder_word, spkr, file)
                            # Get target correct
                            # Accept pairs
                            other_files = [f for f in files if f != file]
                            accept_pairs = [['1', current_file, pattern % (loop, folder_word, spkr, o_file)] for o_file in other_files]

                            # Reject pairs
                            reject_pairs = []
                            n_pairs = len(files)
                            n_target_wrong = n_pairs // 2
                            n_imposters = n_pairs - n_target_wrong
                            
                            tw_all =list(set(glob.glob(root_dir+'/loop%s/*/%s/*.wav' % (loop, spkr))) -
                                set(glob.glob(root_dir+'/loop%s/%s/%s/*.wav' % (loop, folder_word, spkr))))
                            tw_set = random.choices(tw_all, k=n_target_wrong)
                            for tw_sample in tw_set:
                                reject_pairs.append(['0', current_file, tw_sample[len(root_dir)+1:]])

                            imposter_set = random.choices(imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])

                            
                            pairs[phase][gender].extend([accept_pairs] + [reject_pairs])

        # df_bg = pd.DataFrame(background)
        # df_bg.to_csv(os.path.join(root_dir,'loop%s/%s/background.csv'%(loop,folder_word)), index=False)

        for phase in pairs:
            files = pairs[phase]
            for gender in files:
                fout = open(os.path.join(root_dir,'loop%s/%s/%s_%s.txt'%(loop,folder_word,gender,phase)), 'w')
                for list_file in files[gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')

        


## Generate verification file for real dataset

In [10]:
speakers = {
    # 'female': ['female1', 'female2', 'female3', 'female4'],
    'male':['si', 'hung', 'duy', 'kha']
}
# all_speakers = set(speakers['female'] + speakers['male'])
words = [str(i) for i in range(0,10)]
for loop in range(1,6):
    for word in words:
        folder_word = word + ('_' + word) * (loop-1)
        pairs = {'eval': {
                # 'female':[],
                'male':[],
                # 'all':[],
            }}
        for phase in pairs:
            for gender in speakers:
                for spkr in speakers[gender]:
                    imposters = set(speakers[gender]) - set([spkr])
                    imposter_files = []
                    for imp in imposters:
                        imp_file = glob.glob('my_audio_mnist_0_9'+('/loop%s/*/%s/*.wav')%(loop, imp))
                        imposter_files.extend(imp_file)


                    # # Get all 
                    # all_imposters = all_speakers - set([spkr])
                    # all_imposter_files = []
                    # for imp in all_imposters:
                    #     imp_file = glob.glob('my_audio_mnist_0_9'+('/loop%s/*/%s/*.wav')%(loop, imp))
                    #     all_imposter_files.extend(imp_file)

                    path = os.path.join('my_audio_mnist_0_9','loop%s'%(loop), folder_word , spkr)
                    files = os.listdir(path)
                    for file in files:
                        current_file = 'loop%s/%s/%s/%s' % (loop,folder_word, spkr, file)
                        # Get target correct
                        # Accept pairs
                        other_files = [f for f in files if f != file]
                        accept_pairs = [['1', current_file, 'loop%s/%s/%s/%s' % (loop,folder_word, spkr, o_file)] for o_file in other_files]

                        # Reject pairs
                        reject_pairs = []
                        all_reject_pairs = []
                        # n_pairs = len(files)
                        n_pairs = len(files) -1
                        n_target_wrong = n_pairs // 2
                        n_imposters = n_pairs - n_target_wrong
                        
                        tw_all =list(set(glob.glob('my_audio_mnist_0_9'+'/loop%s/*/%s/*.wav' % (loop, spkr))) -
                                set(glob.glob('my_audio_mnist_0_9'+'/loop%s/%s/%s/*.wav' % (loop, folder_word, spkr))))
                        tw_set = random.choices(tw_all, k=n_target_wrong)
                        for tw_sample in tw_set:
                            reject_pairs.append(['0', current_file, tw_sample[len('my_audio_mnist_0_9')+1:]])

                        imposter_set = random.choices(imposter_files, k=n_imposters)
                        for imposter_sample in imposter_set:
                            reject_pairs.append(['0', current_file, imposter_sample[len('my_audio_mnist_0_9')+1:]])
                        pairs[phase][gender].extend([accept_pairs] + [reject_pairs])


                        # imposter_set = random.choices(all_imposter_files, k=n_imposters)
                        # for imposter_sample in imposter_set:
                        #     all_reject_pairs.append(['0', current_file, imposter_sample[len('my_audio_mnist_0_9')+1:]])
                        # pairs['all'].extend([accept_pairs] + [all_reject_pairs])

                        # pairs.extend(accept_pairs)
                # print(pairs)
                # fout = open(os.path.join('my_audio_mnist','%s/pairs_2.txt'%(folder_word)), 'w')
                # for file in pairs:
                #     fout.write(" ".join(file) + '\n')
        for phase in pairs:
            for gender in pairs[phase]:
                fout = open(os.path.join('my_audio_mnist_0_9','loop%s/%s/%s_pairs_fake.txt'%(loop,folder_word,gender)), 'w')
                # print(pairs[gender])
                for list_file in pairs[phase][gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')
                    

In [7]:
# real - fake
speakers = {
    # 'female': ['female1', 'female2', 'female3', 'female4'],
    'male':['si', 'hung', 'duy', 'kha']
}
# all_speakers = set(speakers['female'] + speakers['male'])
words = [str(i) for i in range(0,10)]
for loop in range(1,6):
    for word in words:
        folder_word = word + ('_' + word) * (loop-1)
        pairs = {'eval': {
                # 'female':[],
                'male':[],
                # 'all':[],
            }}
        for phase in pairs:
            for gender in speakers:
                for spkr in speakers[gender]:
  
                    path = os.path.join('my_audio_mnist_0_9','loop%s'%(loop), folder_word , spkr)
                    files = os.listdir(path)
                    for file in files:
                        current_file = 'my_audio_mnist_0_9/loop%s/%s/%s/%s' % (loop,folder_word, spkr, file)
                        # Get target correct
                        # Accept pairs
                        # other_files = [f for f in files if f != file]
                        accept_pairs = [['1', current_file, 'my_audio_mnist_0_9/loop%s/%s/%s/%s' % (loop,folder_word, spkr, o_file)] for o_file in files]

                        # Reject pairs
                        reject_pairs = []
                        all_reject_pairs = []
                        # n_pairs = len(files)
                        n_pairs = len(files) -1
                        # n_target_wrong = n_pairs // 2
                        # n_imposters = n_pairs - n_target_wrong
                        n_fake = n_pairs
                        
                        fake_all =list(set(glob.glob('my_audio_mnist_0_9_fake'+'/loop%s/%s/%s/*.wav' % (loop, folder_word, spkr))))
                        for fake_sample in fake_all:
                            reject_pairs.append(['0', current_file, fake_sample])

                        # imposter_set = random.choices(imposter_files, k=n_imposters)
                        # for imposter_sample in imposter_set:
                        #     reject_pairs.append(['0', current_file, imposter_sample[len('my_audio_mnist_0_9')+1:]])
                        pairs[phase][gender].extend([accept_pairs] + [reject_pairs])


                        # imposter_set = random.choices(all_imposter_files, k=n_imposters)
                        # for imposter_sample in imposter_set:
                        #     all_reject_pairs.append(['0', current_file, imposter_sample[len('my_audio_mnist_0_9')+1:]])
                        # pairs['all'].extend([accept_pairs] + [all_reject_pairs])

                        # pairs.extend(accept_pairs)
                # print(pairs)
                # fout = open(os.path.join('my_audio_mnist','%s/pairs_2.txt'%(folder_word)), 'w')
                # for file in pairs:
                #     fout.write(" ".join(file) + '\n')
        for phase in pairs:
            for gender in pairs[phase]:
                fout = open(os.path.join('my_audio_mnist_0_9','loop%s/%s/%s_pairs_fake.txt'%(loop,folder_word,gender)), 'w')
                # print(pairs[gender])
                for list_file in pairs[phase][gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')
                    

In [5]:
list(set(glob.glob('my_audio_mnist_0_9_fake'+'/loop%s/%s/%s/*.wav' % (1, 0, 'duy'))))

['my_audio_mnist_0_9_fake/loop1/0/duy/0_2.wav',
 'my_audio_mnist_0_9_fake/loop1/0/duy/0_5.wav',
 'my_audio_mnist_0_9_fake/loop1/0/duy/0_3.wav',
 'my_audio_mnist_0_9_fake/loop1/0/duy/0_1.wav',
 'my_audio_mnist_0_9_fake/loop1/0/duy/0_0.wav',
 'my_audio_mnist_0_9_fake/loop1/0/duy/0_7.wav',
 'my_audio_mnist_0_9_fake/loop1/0/duy/0_6.wav',
 'my_audio_mnist_0_9_fake/loop1/0/duy/0_8.wav',
 'my_audio_mnist_0_9_fake/loop1/0/duy/0_9.wav',
 'my_audio_mnist_0_9_fake/loop1/0/duy/0_4.wav']

In [36]:
# words = [str(i) for i in range(0,10)]
# path = 'my_audio_mnist_0_9'
# for loop in range(1,6):
#     folder_loop = 'loop%s'%(loop)
#     for word in words:
#         folder_word = word + ('_' + word) * (loop -1)
        
#         files = glob.glob(os.path.join(path, folder_loop, folder_word,'hung','*.wav'))
#         for i,file in enumerate(files):
#             splits = file.split('/')
#             new_name = folder_word + '_' + str(i)
#             # print(file, '/'.join(splits[:-1]) + '/' + new_name + '.wav')
#             os.rename(file, '/'.join(splits[:-1]) + '/' + new_name + '.wav')


